# Importing Data

In [6]:
# import libraries
from bs4 import BeautifulSoup
from urllib import request
import time
import random
import numpy as np
import pandas as pd
import re

In this file, I will make the previous web scraping codes into a pipelin.

1. functionget_hyperlinks will created a csv file with the name: component_name_hyperlinks.csv. This file stores all the hyperlinks of each item in the search page of url from newegg website.
   a. component_name is the name of the computer components i.e intel_cpu
   b. The url should be in the following form url = "https://www.newegg.com/p/pl?N=100007671%20601306860&Page={}"
   c. num_pages is the total number of pages to be web scraped in the searching page of the url at newegg website
   
2. 

## Web Scraping all Hyperlinks

In [1]:
def get_hyperlinks(component_name,url,num_pages):
    ## create a csv table to store all the hyperlinks of CPU
    filename = "{}_hyperlinks.csv".format(component_name)
    f = open(filename, "w", encoding='utf-8')
    headers = "product_detail,hyperlink\n"
    f.write(headers)
    
    
    # Scraping num_pages of my_url from newegg

    for i in range(1, num_pages+1):
        my_url = url.format(i)
        uClient = request.urlopen(my_url)
        page_html = uClient.read()
        uClient.close()
        page_soup = BeautifulSoup(page_html, "html.parser")
        containers = page_soup.findAll("div", {"class": "item-container"})

        for container in containers:
            ## Find hyperlink that directs to the webpage of that particular product
            hyperlink = list(container.children)[3]['href']

            product_detail = list(container.children)[3].img["alt"].replace(",", "  ")


            f.write(product_detail+","+hyperlink+"\n")

            time.sleep(random.random()+0.5)
        time.sleep(10+random.random())
        print(i)
    
    f.close()

## Get all specs from the hyperlink_file 

In [2]:
## return a list of specs by webscraping all the hyperlinks in the 
#  component_name+"_hyperlinks.csv" file created by get_hyperlinks function 
def get_specs(component_name):
    hyperlinks=pd.read_csv(component_name+"_hyperlinks.csv")
    
    ## all cpu specs will be stored in the list total_specs
    ## then total_specs will be stored in a csv table
    total_specs=[]
    i=0
    
    for url in hyperlinks['hyperlink']:
        uClient = request.urlopen(url)
        page_html = uClient.read()
        uClient.close()
        page_soup = BeautifulSoup(page_html, "lxml")  

        Specs={}
        specs = page_soup.find("div", {"id": "Specs"})
        fieldsets=specs.findAll("fieldset")
        for fieldset in fieldsets:
            dls=fieldset.findAll("dl")
            for dl in dls:
                Specs[dl.find("dt").text]=dl.find("dd").text

        total_specs.append(Specs)
        i+=1
        print(i)

        time.sleep(10+random.random())
        
    return total_specs

In [3]:
## write all the specs into a csv file called component_name+"_specs.csv"
def write_specs(total_specs,component_name):
    ## find the maximal number of header titles for the csv file
    headers=total_specs[0]
    for i in total_specs[1:]:
        for j in i.keys():
            if j not in headers:
                headers[j]=None    
    
    ## make a string file for the header
    headerstr=''
    for i in list(headers.keys()):
        headerstr=headerstr+i+','
    headerstr=headerstr+'\n'      
    
    ## We create a csv table to store all the hyperlinks of CPU
    filename = component_name+"_specs.csv"
    f = open(filename, "w", encoding='utf-8')
    f.write(headerstr)
    
    ## write specs information into the csv file
    headers_list=list(headers.keys())
    for item in total_specs:
        item_str=''
        for col in headers_list:
            item_str=item_str+str(item.get(col,'None')).replace(',',' ')+','
        item_str=item_str[:-1]
        item_str=item_str+'\n'
        f.write(item_str) 
    
    f.close()

In [4]:
## Here is the total pipeline
def pipeline(component_name,url,num_pages):
    get_hyperlinks(component_name,url,num_pages)
    total_specs=get_specs(component_name)
    write_specs(total_specs,component_name)

In [7]:
url='https://www.newegg.com/p/pl?N=100007671%20601306869&Page={}'
component_name='amd_cpu'
num_pages=4
pipeline(component_name,url,num_pages)

1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
